In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import ollama



In [ ]:
"""
export HUGGING_FACE_HUB_TOKEN="<secret token>"
vllm serve 'meta-llama/Meta-Llama-3.1-405B-Instruct' --device=cpu

# Deploy with docker on Linux:
docker run --runtime nvidia --gpus all \
	--name vllm_phi3_steelhacks \
	-v ~/.cache/huggingface:/root/.cache/huggingface \
 	--env "HUGGING_FACE_HUB_TOKEN=<secret token>" \
	-p 8000:8000 \
	--ipc=host \
	vllm/vllm-openai:latest \
	--model microsoft/Phi-3-mini-4k-instruct

# Load and run the model:
docker exec -it vllm_phi3_steelhacks bash -c "vllm serve microsoft/Phi-3-mini-4k-instruct"

# Call the server using curl:
curl -X POST "http://localhost:8000/v1/chat/completions" \ 
	-H "Content-Type: application/json" \ 
	--data '{
		"model": "microsoft/Phi-3-mini-4k-instruct"
		"messages": [
			{"role": "user", "content": "Hello!"}
		]
	}'

docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama 
docker exec -it ollama ollama run llama3.1:8b


    
"""


In [12]:
def make_prompt(fname,user_query):
    with open(f'../prompts/{fname}.txt','r') as pf1:
        return pf1.read().format(**{
            'user-query':user_query
        })

In [20]:
def get_LLM_response(llm_model,fname,user_query):

    prompt = make_prompt(fname,user_query)
    
    msg_to_llm = [
        {
            "role":"system",
            "content":"Your role is rewrite natural language queries into parameter maps to use in API calls."
        }
        {
            "role":"user",
            "content": prompt
        }
    ]

    response = ollama.chat(
        model=llm_model,#"codellama",
        # model = "mistral",
        messages=msg_to_llm,
        stream=False,
        options={
            "temperature":0,
            # "repeat_penalty":1.5,
            # "max_tokens":1024
            # "top_p":1,
            # "frequency_penalty":0,
            # "presence_penalty":0
        }
        
    ) 
    
    ollama.generate(model=llm_model,prompt=prompt)
    
    return response['message']['content']
    # return response['response']


In [16]:
print(make_prompt('prompt1',"test"))

You are provided a user query in free text form search. 
Convert this into a list of parameters and their corresponding values in JSON format where indicated as "user parameters". 
After generating these parameters, then generate a second JSON
with descriptions to these parameters where indicated as "parameter descriptions". 
Be brief and concise.
Generate only where indicated. 
DO NOT generate any free text ONLY JSON.

test

user parameters:
parameter descriptions:





In [24]:
user_query = """
I know JAVA but I don't know python, I am not sure what course to take next which covers python in sufficient depth but 
also covers other things I could do as a programmer I have an A in most courses

"""
# user_query = ' '.join(str(input()).split())
print(f"Requested free text user query:\n{user_query}")



Requested free text user query:

I know JAVA but I don't know python, I am not sure what course to take next which covers python in sufficient depth but 
also covers other things I could do as a programmer I have an A in most courses




In [27]:
response = get_LLM_response('llama3:8b','prompt1',user_query)

In [28]:
print(response)

Here are the JSON outputs:

**User Parameters**
```
[
  {
    "name": "query",
    "value": "I know JAVA but I don't know python, I am not sure what course to take next which covers python in sufficient depth but also covers other things I could do as a programmer I have an A in most courses"
  }
]
```

**Parameter Descriptions**
```
[
  {
    "name": "query",
    "description": "The user's search query in free text form."
  }
]
```
